#**Consultas anidadas (Subqueries).**

Las consultas anidadas son una herramienta flexible y podemos en el cual se pueden utilizar en 4 clausulas principales de SQL (SELECT, INSERT, UPDATE, DELETE). Una consulta anidada corresponde a una subconsulta contenida en otra sentencia SQL, donde esta última llamaremos la sentencia contenedora. Las subconsultas están siempre contenidas dentro de paréntesis y usualmente son ejecutadas previamente a la sentencia contenedora. Estas pueden retornarnos un conjunto los cuales pueden tener:

- Una fila y una columna
- Múltiples filas y una columna
- Múltiples filas y múltiples columnas

Según el conjunto retornado por la subconsulta, podernos determinar cómo pueden ser utilizadas y que operadores de la sentencia contenedora, pueden interactuar con los datos retornados. Cuando la sentencia contenedora es ejecutada en su totalidad, todos los valores retornados por una subconsulta **son descartados**

La sintaxis de una consulta anidada corresponde a la siguiente:

```SQL
SELECT columna1, columna2, ..., columnaN
FROM tabla_A
WHERE columna1 = (
    SELECT columna1
    FROM tabla_B
);
```

En este caso tenemos la consulta contenedora como:

```SQL
SELECT columna1, columna2, ..., columnaN
FROM tabla_A
WHERE columna1 =
```

y la subconsulta como:
```SQL
 (
    SELECT columna1
    FROM tabla_B
);
```


<center><img src="https://drive.google.com/uc?id=1fKhtlmyFD2Vryk3ubFX4g6FsS0PNLNaZ" alt="drawing" width=800px/></center>

In [ ]:
# prompt: crea el código para conectar con drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import sqlite3 #Importamos la libreria
import pandas as pd
#Generamos la conexion
conn = sqlite3.connect("/content/drive/MyDrive/06 SQL/C04/Chinook_Sqlite.sqlite")
cursor = conn.cursor()

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn)

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


Examinaremos la estructura de la tabla Customer.

In [ ]:
pd.read_sql("PRAGMA table_info('Customer')",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,CustomerId,INTEGER,1,None,1
1,1,FirstName,NVARCHAR(40),1,None,0
2,2,LastName,NVARCHAR(20),1,None,0
3,3,Company,NVARCHAR(80),0,None,0
4,4,Address,NVARCHAR(70),0,None,0
5,5,City,NVARCHAR(40),0,None,0
6,6,State,NVARCHAR(40),0,None,0
7,7,Country,NVARCHAR(40),0,None,0
8,8,PostalCode,NVARCHAR(10),0,None,0
9,9,Phone,NVARCHAR(24),0,None,0


Ahora, pensemos en una primera consulta anidada donde queremos obtener el CustomerId, el FirstName y el LastName del cliente con mayor valor de CustomerId (el último cliente de la tabla Customer). Antes de la consulta anidada lo haremos en dos partes usandos dos consultas:

Primero obtenemos el MAX(cutomerId):

In [ ]:
sqlQuery = """
       SELECT MAX(customerId)
       FROM customer
"""
pd.read_sql(sqlQuery,conn)

,MAX(customerId)
0,59


usamos el valor retornado en la consulta anterior como "parámetro" para la siguiente:

In [ ]:
sqlQuery = """
    SELECT customerId,
       firstName,
       lastName
    FROM customer
    WHERE customerId = 59
"""
pd.read_sql(sqlQuery,conn)

,CustomerId,FirstName,LastName
0,59,Puja,Srivastava


Con una subquery podemos generar la cláusula contenedora y la cláusula anidada la cual retorne el valor `MAX(customerId)` de inmediato:

In [ ]:
sqlQuery = """
    SELECT customerId,
       firstName,
       lastName
    FROM customer
    WHERE customerId = (
       SELECT MAX(customerId)
       FROM customer
       );
"""
pd.read_sql(sqlQuery,conn)

,CustomerId,FirstName,LastName
0,59,Puja,Srivastava


Alternativamente y ya que estamos trabajando con SQLite, podemos utilizar funciones de agregación y obtener dicho valor. Pero **recordar** que la siguiente consulta solo funciona en SQLite (por la propiedad de permitir columnas agrupadas y no agrupadas a la vez), en otros motores de base de datos, **La siguiente consulta se debería caer**. Por lo mismo una alternativa a esta consulta la cual si funciona, corresponde a la consulta anterior.

In [ ]:
sqlQuery = """
    SELECT MAX(customerId),
       firstName,
       lastName
    FROM customer
"""
pd.read_sql(sqlQuery,conn)

,MAX(customerId),FirstName,LastName
0,59,Puja,Srivastava


Veamos otro ejemplo utilizando la tabla Track, en este caso, seleccionemos todas las canciones pertenecientes al disco 'Ride The Lightning':

In [ ]:
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    WHERE albumid = (
       SELECT albumid
       FROM album
       WHERE title = 'Ride The Lightning'
    );
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1874,Fight Fire With Fire,154
1,1875,Ride The Lightning,154
2,1876,For Whom The Bell Tolls,154
3,1877,Fade To Black,154
4,1878,Trapped Under Ice,154
5,1879,Escape,154
6,1880,Creeping Death,154
7,1881,The Call Of Ktulu,154


Alternativamente, nosotros podríamos escribir la consulta anidada, utilizando joins y luego haciendo la cláusula `WHERE` para obtener el nombre de album.

**Nota:** No siempre podremos traspasar una consulta anidada a una consulta utilizando joins (de una manera tan sencilla como este ejemplo). Todo va a depender de la complejidad de las subconsultas.


In [ ]:
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    NATURAL JOIN Album a
    WHERE a.title = 'Ride The Lightning'
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1874,Fight Fire With Fire,154
1,1875,Ride The Lightning,154
2,1876,For Whom The Bell Tolls,154
3,1877,Fade To Black,154
4,1878,Trapped Under Ice,154
5,1879,Escape,154
6,1880,Creeping Death,154
7,1881,The Call Of Ktulu,154


#**Tipos de consultas anidadas**

Existen dos tipos de consultas anidadas: **las correlacionadas** y las **no correlacionadas**. Hasta ahora, todas las consultas que hemos visto corresponden a consultas **no correlacionadas**, las cuales se caracterizan por no referenciar nada de la consulta contenedora. Por otra parte, las consultas anidadas **correlacionadas** hacen referencia a la consulta contenedora, pero esto lo vamos a ver más adelante.

Otro tipo de consultas anidadas, pueden ser las **consultas escalares**, el resultado de la subconsulta es simplemente una fila y una columna, otro ejemplo de una consulta escalar:


In [ ]:
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    WHERE albumid != (
       SELECT albumid
       FROM album
       WHERE title = 'Ride The Lightning'
    );
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1,For Those About To Rock (We Salute You),1
1,2,Balls to the Wall,2
2,3,Fast As a Shark,3
3,4,Restless and Wild,3
4,5,Princess of the Dawn,3
...,...,...,...
3490,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343
3491,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344
3492,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345
3493,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346


Las subconsultas **no escalares**, también conocidas como subconsultas que tienen múltiples filas, requieren ciertos operadores para poder hacer la comparación. En la mayoría de los motores de base de datos no podemos hacer una consulta del siguiente tipo:

```SQL
SELECT trackid,
   name,
   albumid
FROM track
WHERE albumid = (
   SELECT albumid
   FROM album
   WHERE title <> 'Ride The Lightning'
);
```
La subconsulta retorna un subconjunto de identificadores, y técnicamente no podemos igualar un identificador a un conjunto de identificadores, por lo que tendríamos que utilizar la sentencia **IN** (o su negación **NOT IN**).


In [ ]:
#Utilizando la sintaxis correcta
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    WHERE albumid IN (
       SELECT albumid
       FROM album
       WHERE title <> 'Ride The Lightning'
    );
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1,For Those About To Rock (We Salute You),1
1,6,Put The Finger On You,1
2,7,Let's Get It Up,1
3,8,Inject The Venom,1
4,9,Snowballed,1
...,...,...,...
3490,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343
3491,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344
3492,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345
3493,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346


In [ ]:
# Veamos que retorna la subconsulta
sqlQuery = """
   SELECT albumid
   FROM album
   WHERE title <> 'Ride The Lightning'
"""
pd.read_sql(sqlQuery,conn)

,AlbumId
0,1
1,2
2,3
3,4
4,5
...,...
341,343
342,344
343,345
344,346


**¿Cual es el resultado de la sgte consulta?**

In [ ]:
sqlQuery = """
    SELECT trackid,
       name,
       albumid
    FROM track
    WHERE albumid NOT IN (
       SELECT albumid
       FROM album
       WHERE title <> 'Ride The Lightning'
    );
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,1874,Fight Fire With Fire,154
1,1875,Ride The Lightning,154
2,1876,For Whom The Bell Tolls,154
3,1877,Fade To Black,154
4,1878,Trapped Under Ice,154
5,1879,Escape,154
6,1880,Creeping Death,154
7,1881,The Call Of Ktulu,154


LOS CLIENTES QUE TIENEN EL MÁXIMO DE FACTURA EN TOTAL

In [ ]:
# TODA FUNCIÓN DEBE VENIR DENTRO DE UN SELECT
sqlQuery = """
    SELECT *
    FROM Customer
    WHERE CustomerId IN (
        SELECT CustomerId
        FROM Invoice
        WHERE Total = (
            SELECT MAX(Total)
            FROM Invoice
        )
    );
"""

pd.read_sql(sqlQuery,conn)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId,Premium
0,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5,1


# **Subconsultas correlacionadas**

Las subconsultas correlacionadas, tienen explícitamente una condición entre la subconsulta y la consulta contenedora. Cabe detallar que, si nosotros utilizamos subconsultas correlacionadas, la subconsulta se va a ejecutar por cada fila candidata que pueda cumplir las condiciones.

Veamos un ejemplo obteniendo todas las canciones las cuales estén exactamente en 5 playlists:


In [ ]:
sqlQuery = """
    SELECT count(*)
    FROM PlayListTrack
  where trackid = 3492;
"""

pd.read_sql(sqlQuery,conn)

,count(*)
0,5


In [ ]:
sqlQuery = """
    SELECT t.TrackId,
       name,
       albumid
    FROM Track t
    WHERE (
       SELECT COUNT(*)
       FROM PlayListTrack plt
       WHERE plt.TrackId = t.TrackId
    ) = 5 ;
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,3403,Intoitus: Adorate Deum,272
1,3404,"Miserere mei, Deus",273
2,3408,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",277
3,3409,"Suite for Solo Cello No. 1 in G Major, BWV 100...",278
4,3410,"The Messiah: Behold, I Tell You a Mystery... T...",279
5,3411,Solomon HWV 67: The Arrival of the Queen of Sheba,280
6,3415,Symphony No.5 in C Minor: I. Allegro con brio,284
7,3416,Ave Maria,285
8,3417,"Nabucco: Chorus, ""Va, Pensiero, Sull'ali Dorate""",286
9,3418,Die Walküre: The Ride of the Valkyries,287


Qué calcula la siguiente consulta?:

In [ ]:
sqlQuery = """
    SELECT t.TrackId,
       name,
       albumid,
       (
       SELECT COUNT(*)
       FROM PlayListTrack plt
       WHERE plt.TrackId = t.TrackId
       ) as counts
    FROM Track t
    ;
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,counts
0,1,For Those About To Rock (We Salute You),1,3
1,2,Balls to the Wall,2,3
2,3,Fast As a Shark,3,4
3,4,Restless and Wild,3,4
4,5,Princess of the Dawn,3,4
...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,5
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,4
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,4
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,4


La consulta anterior calcula el número de PlayLists en que aparece cada Canción.

El operador EXISTS nos permite obtener la comprobación de que una relación existe, independiente de la cantidad que existan al momento de hacer la subconsulta. Esto ocurre ya sea si seleccionamos una, muchas o ninguna de las columnas presentes en una tabla.

In [ ]:
sqlQuery = """
    SELECT t.TrackId,
       t.name,
       t.albumid
    FROM Track t
    INNER JOIN Genre g
    USING(GenreId)
    WHERE EXISTS
    (
        SELECT 1 FROM PlaylistTrack plt
        WHERE plt.TrackId = t.Trackid
            AND g.Name = 'Blues'
    )
    ;
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId
0,194,First Time I Met The Blues,20
1,195,Let Me Love You Baby,20
2,196,Stone Crazy,20
3,197,Pretty Baby,20
4,198,When My Left Eye Jumps,20
...,...,...,...
76,2586,Twice As Hard,210
77,2587,Lickin',210
78,2588,Soul Singing,210
79,2589,Hard To Handle,210


Veamos qué es lo que nos retorna la consulta Exists:

In [ ]:
sqlQuery = """
    SELECT t.TrackId,
       EXISTS
    (
        SELECT 1 FROM PlaylistTrack plt
        WHERE plt.TrackId = t.Trackid
            AND g.Name = 'Blues'
    ) as result_exists
    FROM Track t
    INNER JOIN Genre g
    USING(GenreId)
    ORDER BY 2 DESC
    ;
"""

pd.read_sql(sqlQuery,conn)

,TrackId,result_exists
0,194,1
1,195,1
2,196,1
3,197,1
4,198,1
...,...,...
3498,3499,0
3499,3500,0
3500,3501,0
3501,3502,0


#**Actividad 1**

1. Encuentre el nombre de la canción con mayor duración. Para verificación, la duración en milisegundos es de 5.286.953.

2. Encuentre la canción con mayor duración del album cuyo título es 'War' del grupo 'U2'.

3. Obtenga todas las canciones las cuales nunca se han comprado. Debe retornar 1519 canciones. Para esto utilice IN y Exists (en dos consultas separadas).

4. Obtener una lista de los primeros 10 nombres de canciones, de la playlist 'TV Shows', ordénelos de forma alfabéticamente.

5. Encuentre cuales de las playlist tienen la mayor duración (Puede ser mas de 1 playlist).

6. **Existen otros dos operadores que pueden utilizar, los cuales son el `ANY` y el `ALL`. Averigüe que es lo que realizan cada uno de estos, y proponga dos consultas con la BD Chinook**


##**Consulta 1**

In [ ]:
# Su código aquí (SE COMPRUEBA LA DURACIÓN)
# Encuentre el nombre de la canción con mayor duración. Para verificación, la duración en milisegundos es de 5.286.953.
sqlQuery = """
    SELECT trackid, name,
       MAX(Milliseconds)
    FROM track
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,MAX(Milliseconds)
0,2820,Occupation / Precipice,5286953


##**Consulta 2**

In [ ]:
# Su código aquí
#Encuentre la canción con mayor duración del album cuyo título es 'War' del grupo 'U2'.
sqlQuery = """
    SELECT trackid, name, Milliseconds as Duracion
    FROM track
    WHERE Milliseconds = (
       SELECT MAX(Milliseconds)
       FROM track
       WHERE Albumid =(
        SELECT albumid
        FROM album
        WHERE title = 'War' AND artistid = (
          SELECT artistid
          FROM artist
          WHERE name = 'U2')
        )
    );
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,Duracion
0,3020,New Year's Day,336274


##**Consulta 3**

Obtenga todas las canciones las cuales nunca se han comprado. Debe retornar 1519 canciones. Para esto utilice IN y Exists (en dos consultas separadas).

In [ ]:
# Su código aquí
# Obtenga todas las canciones las cuales nunca se han comprado. Debe retornar 1519 canciones. Para esto utilice IN y Exists (en dos consultas separadas).
sqlQuery = """
    SELECT trackid, name
    FROM track
    WHERE trackid NOT IN (
       SELECT trackid
       FROM invoiceline
       );
"""

pd.read_sql(sqlQuery,conn)


,TrackId,Name
0,7,Let's Get It Up
1,11,C.O.D.
2,17,Let There Be Rock
3,18,Bad Boy Boogie
4,22,Whole Lotta Rosie
...,...,...
1514,3497,"Erlkonig, D.328"
1515,3498,"Concerto for Violin, Strings and Continuo in G..."
1516,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)"
1517,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


In [ ]:
sqlQuery_EXISTS = """
    SELECT trackid, name
    FROM Track t
    WHERE NOT EXISTS (
        SELECT 1
        FROM InvoiceLine il
        WHERE il.TrackId = t.TrackId
    );
"""
pd.read_sql(sqlQuery_EXISTS, conn)


,TrackId,Name
0,7,Let's Get It Up
1,11,C.O.D.
2,17,Let There Be Rock
3,18,Bad Boy Boogie
4,22,Whole Lotta Rosie
...,...,...
1514,3497,"Erlkonig, D.328"
1515,3498,"Concerto for Violin, Strings and Continuo in G..."
1516,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)"
1517,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


##**Consulta 4**

Obtener una lista de los primeros 10 nombres de canciones, de la playlist 'TV Shows', ordénelos de forma alfabéticamente.

In [ ]:
# Su código aquí
# Obtener una lista de los primeros 10 nombres de canciones, de la playlist 'TV Shows', ordénelos de forma alfabéticamente.
sqlQuery = """
    SELECT t.name
    FROM track t
    WHERE t.trackid IN (
        SELECT plt.trackid
        FROM playlisttrack plt
        WHERE plt.playlistid = (
            SELECT p.playlistid
            FROM playlist p
            WHERE p.name = 'TV Shows'
            )
        )
    ORDER BY t.name
    LIMIT 10
"""

pd.read_sql(sqlQuery,conn)

,Name
0,"""?"""
1,...And Found
2,...In Translation
3,.07%
4,"A Benihana Christmas, Pts. 1 & 2"
5,A Day In the Life
6,A Measure of Salvation
7,A Tale of Two Cities
8,Abandoned
9,Adrift


##**Consulta 5**

In [ ]:
#Su código aquí
# Encuentre cuales de las playlist tienen la mayor duración (Puede ser mas de 1 playlist).
sqlQuery = """
SELECT Playlistid as ID, Name AS Nombre,
       (SELECT SUM(Milliseconds)
        FROM Track
        WHERE TrackId IN (SELECT TrackId FROM PlaylistTrack WHERE PlaylistId = p.PlaylistId)
       ) AS TotalDuration
FROM Playlist p
WHERE (SELECT SUM(Milliseconds)
       FROM Track
       WHERE TrackId IN (SELECT TrackId FROM PlaylistTrack WHERE PlaylistId = p.PlaylistId)
      ) = (SELECT MAX(TotalDuration)
           FROM (SELECT PlaylistId,
                        (SELECT SUM(Milliseconds)
                         FROM Track
                         WHERE TrackId IN (SELECT TrackId FROM PlaylistTrack WHERE PlaylistId = pt.PlaylistId)
                        ) AS TotalDuration
                 FROM PlaylistTrack pt
                 GROUP BY PlaylistId
                )
    )
    ;
"""

pd.read_sql(sqlQuery,conn)

,ID,Nombre,TotalDuration
0,1,Music,877683083
1,8,Music,877683083


Existen 2 Playlists que cumplen con la condición. Tienen el mismo nombre (**Music**), y **sus ID son 1 y 8**.

##**Punto 6**
Existen otros dos operadores que pueden utilizar, los cuales son el ANY y el ALL. Averigüe que es lo que realizan cada uno de estos, y proponga dos consultas con la BD Chinook

Los operadores ANY y ALL se usan en consultas SQL para comparar un valor con un conjunto de valores devueltos por una subconsulta.

**ANY (Cualquiera de los valores)**

*   Devuelve TRUE si al menos un valor de la subconsulta cumple la condición.
*   Se usa con operadores como =, >, <, >=, <=, <>.
*   Equivalente a usar IN cuando se usa con =, pero permite otros operadores.

**ALL (Todos los valores)**

*   Devuelve TRUE solo si la condición se cumple para todos los valores de la subconsulta.
*   Se usa típicamente con >, <, >=, <=, <>.

Algunos ejemplos serían:

a) Obtener las canciones (TrackId, Name) cuyo tiempo de duración (Milliseconds) sea mayor que todas las canciones de género Jazz.

`SELECT TrackId, Name
FROM Track
WHERE Milliseconds > ALL (
    SELECT Milliseconds
    FROM Track
    WHERE GenreId = (
        SELECT GenreId
        FROM Genre
        WHERE Name = 'Jazz'
        )
    );`


b) Obtener los clientes (FirstName, LastName) que han gastado más dinero en una sola factura que cualquier otro cliente con CustomerId = 3.

`SELECT FirstName, LastName
FROM Customer
WHERE CustomerId IN (
    SELECT CustomerId
    FROM Invoice
    WHERE Total > ANY (
        SELECT Total
        FROM Invoice
        WHERE CustomerId = 3
    )
);`


Cuando se intentó ejecutar, no funcionó dando mensajes de error que podrían deberse a que SQLite no soportaría directamente los operadores `ANY` y `ALL`.


#**Fin Actividad 1**

#**Inserción, Modificación y eliminación con subconsultas.**

Unas de las características principales de las subconsultas es la versatilidad al poder retornar elementos que nosotros queramos en la subconsulta, y luego manipular dichos resultados en la consulta contenedora. En este caso, nosotros podemos utilizar las sentencias `INSERT`, `UPDATE` y `DELETE` en conjunto con las subconsultas para adecuar según nuestras necesidades las distintas operaciones que vamos a aplicar. En primer lugar, vamos a hacer un respaldo de la tabla Tracks, ya que vamos a realizar operaciones modificando dicha tabla. Para esto, vamos a extraer el código SQL utilizado para crear dicha tabla y generar un duplicado de esta (pero con otro nombre).


**Nota:** en otros motores de base de datos, existen operaciones similares para obtener el SQL que crea una tabla, por ejemplo mysql implementa `SHOW CREATE TABLE tbl_name`


In [ ]:
# Seleccionamos todo desde el catalogo, pero especificamente que el nombre de la tabla sea Track y extraemos el SQL que la crea
sql_Track = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table' AND name = 'Track';",conn).sql[0]
print(sql_Track)

CREATE TABLE [Track]
(
    [TrackId] INTEGER  NOT NULL,
    [Name] NVARCHAR(200)  NOT NULL,
    [AlbumId] INTEGER,
    [MediaTypeId] INTEGER  NOT NULL,
    [GenreId] INTEGER,
    [Composer] NVARCHAR(220),
    [Milliseconds] INTEGER  NOT NULL,
    [Bytes] INTEGER,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Track] PRIMARY KEY  ([TrackId]),
    FOREIGN KEY ([AlbumId]) REFERENCES [Album] ([AlbumId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([GenreId]) REFERENCES [Genre] ([GenreId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([MediaTypeId]) REFERENCES [MediaType] ([MediaTypeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


In [ ]:
#Reemplazamos el nombre de dicha tabla, con el fin de que no existan problemas de tablas duplicadas
sql_Track = sql_Track.replace("[Track]","[Track_BKP]")

Recordar que, para escribir cambios a disco, nosotros tenemos que utilizar el método de la conexión `commit`. Todos los cambios sin salvar se escriben en disco y quedan persistentemente para consultas futuras

In [ ]:
#Creamos la tabla Track_BKP
cursor.execute(sql_Track)
conn.commit()

In [ ]:
#Revisamos si esta la nueva tabla que acabamos de crear
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn)

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


Supongamos que queremos respaldar todos los datos de la tabla Track, hacia Track_BKP, esto lo podemos realizar con la siguiente sentencia

In [ ]:
sqlQuery = """
INSERT INTO Track_BKP
    SELECT * FROM Track
"""

cursor.execute(sqlQuery)

Luego podemos consultar si está poblada o no la tabla Track_BKP, haciendo un `SELECT`

In [ ]:
sqlQuery = """
    SELECT *
    FROM Track_BKP
"""
pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


Ahora supongamos que en verdad no queremos guardar todos los registros, sino más bien los que vamos a ir actualizando, con el fin de mantener su estado inicial. Podríamos eliminar todos los registros de nuestra tabla utilizando la sentencia DELETE. Pero, como todavía nosotros no hacemos un commit, podemos hacer una operación alternativa. La operación `rollback()` nos permitirá devolver a su estado original todos los cambios hechos y que no hayan pasado por un commit. En este caso vamos a llamar a `conn.rollback()` para dejar la tabla Track_BKP vacía.

In [ ]:
conn.rollback()

In [ ]:
#Examinamos la tabla
sqlQuery = """
    SELECT *
    FROM Track_BKP
"""
pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice


Con todo esto, ahora hagamos el ejercicio de hacer un descuento para las Canciones de los grupos AC/DC y Metallica. En primer lugar necesitamos obtener todos los IDs de las canciones de ambos grupos:

In [ ]:
sqlQuery = """
    SELECT t.TrackId, a.Name
    FROM Track T
    INNER JOIN Album al
    ON t.AlbumId = al.AlbumId
    INNER JOIN Artist a
    ON al.ArtistId = a.ArtistId
    WHERE a.Name IN ('AC/DC','Metallica')
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name
0,1,AC/DC
1,6,AC/DC
2,7,AC/DC
3,8,AC/DC
4,9,AC/DC
...,...,...
125,1897,Metallica
126,1898,Metallica
127,1899,Metallica
128,1900,Metallica


Ahora utilizando los IDs obtenidos anteriormente, podemos insertar en la tabla de Backup solo las canciones que vamos a modificar:

In [ ]:
sqlQuery = """
INSERT INTO Track_BKP
    SELECT * FROM Track
    WHERE Track.trackId IN
    (
        SELECT t.TrackId from Track T
        INNER JOIN Album al
        ON t.AlbumId = al.AlbumId
        INNER JOIN Artist a
        ON al.ArtistId = a.ArtistId
        WHERE a.Name IN ('AC/DC','Metallica')
    )

"""

cursor.execute(sqlQuery)

In [ ]:
#Revisamos que solo esten canciones de AC/DC y Metallica
sqlQuery = """
    SELECT *
    FROM Track_BKP
"""
pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
...,...,...,...,...,...,...,...,...,...
125,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.99
126,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.99
127,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.99
128,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.99


In [ ]:
#Escribimos en la base de datos los cambios
conn.commit()

Ahora, con los datos respaldados, podremos hacer el proceso de modificar la tabla, primero definiremos la subquery que nos devolverá todos los identificadores para modificar

In [ ]:
subSqlQuery = """
        SELECT t.TrackId
        FROM Track T
        INNER JOIN Album al
        ON t.AlbumId = al.AlbumId
        INNER JOIN Artist a
        ON al.ArtistId = a.ArtistId
        WHERE a.Name IN ('AC/DC','Metallica')
"""

y utilizando `format` podemos incluir fácilmente el string de la subconsulta, en la consulta contenida. En este caso, vamos a bajar un $50\%$ el valor unitario de las canciones de AC/DC y Metallica

In [ ]:
sqlQuery = f"""
    UPDATE Track
    SET UnitPrice = UnitPrice * 0.5
    WHERE Track.trackId IN
    ({subSqlQuery})

"""

cursor.execute(sqlQuery)

Ahora revisamos todas las canciones que tengan un valor menor al mínimo antiguo (que era 0.99) y podemos revisar que son todas canciones de AC/DC o Metallica

In [ ]:

sqlQuery = """
    SELECT *
    FROM Track
    WHERE UnitPrice < 0.99
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.495
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.495
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.495
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.495
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.495
...,...,...,...,...,...,...,...,...,...
125,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.495
126,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.495
127,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.495
128,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.495


En un cambio de espíritu, al gerente de la empresa se dio cuenta que la verdad no le gusta tanto AC/DC para poder hacer el descuento a dicha banda. Por lo mismo nos ha pedido revertir los cambios, pero solo para dicha banda. Entonces nosotros podemos redefinir la subconsulta y aplicar tanto las cláusulas de UPDATE y DELETE donde ocurran las condiciones que nosotros queramos (Que sean de la banda AC/DC).

In [ ]:
subSqlQuery = """
        SELECT t.TrackId from Track t
        INNER JOIN Album al
        ON t.AlbumId = al.AlbumId
        INNER JOIN Artist a
        ON al.ArtistId = a.ArtistId
        WHERE a.Name = 'AC/DC'
"""

#Devolvemos al precio Original, como posible mejora de la bitacora, podemos extraer dicho valor de la tabla de Backup
sqlQuery1 = f"""
    UPDATE Track
    SET UnitPrice = UnitPrice * 2
    WHERE trackId IN
    ({subSqlQuery})
"""

#Eliminamos las canciones que ya no necesitamos Backup
sqlQuery2 = f"""
DELETE FROM Track_BKP
    WHERE trackId IN
    ({subSqlQuery})
"""
cursor.execute(sqlQuery1)
cursor.execute(sqlQuery2)

Ahora vemos que las canciones que quedan, son solo de metallica tanto en precio, como en la tabla de backup

In [ ]:
sqlQuery = """
    SELECT *
    FROM Track
    WHERE UnitPrice < 0.99
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,408,Free Speech For The Dumb,35,1,3,Molaney/Morris/Roberts/Wainwright,155428,5076048,0.495
1,409,It's Electric,35,1,3,Harris/Tatler,213995,6978601,0.495
2,410,Sabbra Cadabra,35,1,3,Black Sabbath,380342,12418147,0.495
3,411,Turn The Page,35,1,3,Seger,366524,11946327,0.495
4,412,Die Die My Darling,35,1,3,Danzig,149315,4867667,0.495
...,...,...,...,...,...,...,...,...,...
107,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.495
108,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.495
109,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.495
110,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.495


In [ ]:
sqlQuery = """
    SELECT *
    FROM Track_BKP
"""

pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,408,Free Speech For The Dumb,35,1,3,Molaney/Morris/Roberts/Wainwright,155428,5076048,0.99
1,409,It's Electric,35,1,3,Harris/Tatler,213995,6978601,0.99
2,410,Sabbra Cadabra,35,1,3,Black Sabbath,380342,12418147,0.99
3,411,Turn The Page,35,1,3,Seger,366524,11946327,0.99
4,412,Die Die My Darling,35,1,3,Danzig,149315,4867667,0.99
...,...,...,...,...,...,...,...,...,...
107,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.99
108,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.99
109,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.99
110,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.99


In [ ]:
#Guardamos los cambios
conn.commit()

In [ ]:
sqlQuery = """
    SELECT *
    FROM Track_BKP
"""
pd.read_sql(sqlQuery,conn)


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,408,Free Speech For The Dumb,35,1,3,Molaney/Morris/Roberts/Wainwright,155428,5076048,0.99
1,409,It's Electric,35,1,3,Harris/Tatler,213995,6978601,0.99
2,410,Sabbra Cadabra,35,1,3,Black Sabbath,380342,12418147,0.99
3,411,Turn The Page,35,1,3,Seger,366524,11946327,0.99
4,412,Die Die My Darling,35,1,3,Danzig,149315,4867667,0.99
...,...,...,...,...,...,...,...,...,...
107,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.99
108,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.99
109,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.99
110,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.99


#**Actividad 2**

1. Hacer un update, generando un descuento de un 20% para todas las canciones las cuales tengan un valor mayor al promedio. Previamente al update, respaldar las canciones en la tabla de respaldo

2. Genere una tabla de respaldo para la Tabla de los Clientes (Customer). Añada una columna adicional a la tabla Customer Original, llamada premium. Para esto utilice la sentencia que se detalla a continuación **¿Como podría interpretar dicha sentencia?**

```SQL
ALTER TABLE Customer
ADD COLUMN Premium INTEGER NOT NULL DEFAULT 0;
```

3. Con la columna ya generada en la tabla Customer, editar el valor de dicha columna para todos los clientes que hayan gastado más de 40 dolares en compras. Esto se realiza con el fin de poder implementar a futuro un sistema de clientes premium, a los cuales se les otorgara descuentos y regalos.

4. Obtenga el nombre, apellido y país de los clientes premium


##**Consulta 1**

Hacer un update, generando un descuento de un 20% para todas las canciones las cuales tengan un valor mayor al promedio. Previamente al update, respaldar las canciones en la tabla de respaldo

In [ ]:
# Su código aquí
# respaldo
# Obtener el SQL para crear la tabla original Track
sql_Track = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table' AND name = 'Track';", conn).sql[0]

# Reemplazar el nombre de la tabla para crear Track_RESP
sql_Track = sql_Track.replace("Track", "Track_RESP")

# Ejecutar la creación de la nueva tabla Track_Track_RESP
cursor.execute(sql_Track)
conn.commit()

In [ ]:
#Revisamos si esta la nueva tabla que acabamos de crear
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn)

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [ ]:
# Respaldo de los datos
sqlQuery = """
INSERT INTO Track_RESP
    SELECT * FROM Track
"""

cursor.execute(sqlQuery)

In [ ]:
# Revision
sqlQueryy = """
    SELECT *
    FROM Track_RESP
"""
pd.read_sql(sqlQueryy,conn)

,Track_RESPId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [ ]:
#Guardamos los cambios
conn.commit()

In [ ]:
#Hacer un update, generando un descuento de un 20% para todas las canciones las cuales tengan un valor mayor al promedio.
sqlQuery = """
    UPDATE Track
    SET UnitPrice = UnitPrice * 0.8
    WHERE UnitPrice > (SELECT AVG(UnitPrice) FROM Track)
"""

cursor.execute(sqlQuery)

In [ ]:
#Guardamos los cambios
conn.commit()

##**Consulta 2**

In [ ]:
# Su código aquí
# Tabla de respaldo para Customer
sql_Customer = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table' AND name = 'Customer';",conn).sql[0]
print(sql_Customer)


CREATE TABLE [Customer]
(
    [CustomerId] INTEGER  NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),
    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([EmployeeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


In [ ]:
#Reemplazamos el nombre de dicha tabla, con el fin de que no existan problemas de tablas duplicadas
sql_Customer = sql_Customer.replace("[Customer]","[Customer_BKP]")

In [ ]:
#Creamos la tabla Customer_BKP
cursor.execute(sql_Customer)
conn.commit()

In [ ]:
#Revisamos si esta la nueva tabla que acabamos de crear
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn)

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [ ]:
# Respaldo de los datos
sqlQuery = """
INSERT INTO Customer_BKP
    SELECT * FROM Customer
"""

cursor.execute(sqlQuery)

In [ ]:
# Verificamos
sqlQuery = """
    SELECT *
    FROM Customer_BKP
"""
pd.read_sql(sqlQuery,conn)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
5,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
6,7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
7,8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
8,9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
9,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


In [ ]:
#Guardamos los cambios
conn.commit()

In [ ]:
# Añada una columna adicional a la tabla Customer Original, llamada premium. Para esto utilice la sentencia que se detalla a continuación
# ¿Como podría interpretar dicha sentencia?

sqlQuery = """
ALTER TABLE Customer
ADD COLUMN Premium INTEGER NOT NULL DEFAULT 0;
"""

cursor.execute(sqlQuery)

In [ ]:
#Guardamos los cambios
conn.commit()

In [ ]:
pd.read_sql("PRAGMA table_info('Customer')", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,CustomerId,INTEGER,1,None,1
1,1,FirstName,NVARCHAR(40),1,None,0
2,2,LastName,NVARCHAR(20),1,None,0
3,3,Company,NVARCHAR(80),0,None,0
4,4,Address,NVARCHAR(70),0,None,0
5,5,City,NVARCHAR(40),0,None,0
6,6,State,NVARCHAR(40),0,None,0
7,7,Country,NVARCHAR(40),0,None,0
8,8,PostalCode,NVARCHAR(10),0,None,0
9,9,Phone,NVARCHAR(24),0,None,0


Vemos que se modica la estructua de la tabla Customer, agregando un campo llamado "Premium", entero no nulo, y con un valor por defecto de cero.

##**Consulta 3**

In [ ]:
# Su código aquí
# Con la columna ya generada en la tabla Customer, editar el valor de dicha columna para todos los clientes que hayan gastado más de 40 dolares en compras.
# Esto se realiza con el fin de poder implementar a futuro un sistema de clientes premium, a los cuales se les otorgara descuentos y regalos.
#Hacer un update, generando un descuento de un 20% para todas las canciones las cuales tengan un valor mayor al promedio.
sqlQuery = """
    UPDATE Customer
    SET Premium = 1
    WHERE CustomerId IN
      (SELECT CustomerId
      FROM Invoice
      GROUP BY 1
      HAVING SUM(Total) > 40);
"""

cursor.execute(sqlQuery)

In [ ]:
# Verficamos
sqlQuery = """
SELECT CustomerId, SUM(Total)
FROM Invoice
GROUP BY 1
HAVING SUM(Total) > 40;
"""

pd.read_sql(sqlQuery,conn)

,CustomerId,SUM(Total)
0,5,40.62
1,6,49.62
2,7,42.62
3,24,43.62
4,25,42.62
5,26,47.62
6,28,43.62
7,37,43.62
8,43,40.62
9,44,41.62


In [ ]:
# Contrastamos con clientes Premium
sqlQuery = """
SELECT CustomerId, Premium
FROM Customer
WHERe Premium=1;
"""

pd.read_sql(sqlQuery,conn)

,CustomerId,Premium
0,5,1
1,6,1
2,7,1
3,24,1
4,25,1
5,26,1
6,28,1
7,37,1
8,43,1
9,44,1


Se verifican que ambas listas coinciden. Se procede a grabar en disco.

In [ ]:
#Guardamos los cambios
conn.commit()

##**Consulta 4**

In [ ]:
# Su código aquí
# Obtenga el nombre, apellido y país de los clientes premium
sqlQuery = """
SELECT FirstName, LastName, Country
FROM Customer
WHERE Premium=1;
"""

pd.read_sql(sqlQuery,conn)

,FirstName,LastName,Country
0,František,Wichterlová,Czech Republic
1,Helena,Holý,Czech Republic
2,Astrid,Gruber,Austria
3,Frank,Ralston,USA
4,Victor,Stevens,USA
5,Richard,Cunningham,USA
6,Julia,Barnett,USA
7,Fynn,Zimmermann,Germany
8,Isabelle,Mercier,France
9,Terhi,Hämäläinen,Finland


##**Fin Actividad 2**

# <font color='red'>__LINK DE INTERÉS__: Tutorial de apoyo para SQLite</font>

El siguiente link aborda el uso de SQLite, complementando y reforzando los contenidos vistos en clases [aquí](https://www.w3resource.com/sqlite/sqlite-subqueries.php).

## <font color='purple'> __EXPERIMENTO__: </font>
### Obtener los artistas que tienen más álbumes que el promedio de todos los artistas.

In [ ]:
sqlQuery = """
SELECT Artist.Name, COUNT(Album.AlbumId) AS N°_Albumes
FROM Artist
JOIN Album ON Artist.ArtistId = Album.ArtistId
GROUP BY Artist.ArtistId
HAVING COUNT(Album.AlbumId) > (
    SELECT AVG(AlbumCount)
    FROM (
        SELECT COUNT(AlbumId) AS AlbumCount
        FROM Album
        GROUP BY ArtistId
    )
)
ORDER BY 2 DESC;
"""

pd.read_sql(sqlQuery,conn)

,Name,N°_Albumes
0,Iron Maiden,21
1,Led Zeppelin,14
2,Deep Purple,11
3,Metallica,10
4,U2,10
5,Ozzy Osbourne,6
6,Pearl Jam,5
7,Various Artists,4
8,Faith No More,4
9,Foo Fighters,4


### <font color='purple'>Fin experimento </font>

## <font color='purple'> __EXPERIMENTO__: </font>
### Obtener los álbumes más caros de cada género, considerando el precio del album como la suma de los valores de las canciones que lo componen.

In [ ]:
sqlQuery = """
WITH AlbumPrices AS (
    SELECT Album.AlbumId,
           Album.Title AS Album,
           Artist.Name AS Artista,
           Genre.GenreId,
           Genre.Name AS Genero,
           SUM(Track.UnitPrice) AS PrecioTotal
    FROM Album
    JOIN Artist ON Album.ArtistId = Artist.ArtistId
    JOIN Track ON Album.AlbumId = Track.AlbumId
    JOIN Genre ON Track.GenreId = Genre.GenreId
    GROUP BY Album.AlbumId, Genre.GenreId
)
SELECT Album, Artista, Genero, PrecioTotal
FROM AlbumPrices
WHERE PrecioTotal = (
    SELECT MAX(PrecioTotal)
    FROM AlbumPrices AS AP
    WHERE AP.GenreId = AlbumPrices.GenreId
)
ORDER BY Genero;
"""

pd.read_sql(sqlQuery,conn)

,Album,Artista,Genero,PrecioTotal
0,Carry On,Chris Cornell,Alternative,13.860
1,Revelations,Audioslave,Alternative,13.860
2,Acústico,Titãs,Alternative & Punk,21.780
3,The Cream Of Clapton,Eric Clapton,Blues,17.820
4,Vinícius De Moraes - Sem Limite,Toquinho & Vinícius,Bossa Nova,14.850
5,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,Classical,1.980
6,English Renaissance,The King's Singers,Classical,1.980
7,"The Office, Season 3",The Office,Comedy,27.064
8,"Lost, Season 3",Lost,Drama,35.024
9,My Way: The Best Of Frank Sinatra [Disc 1],Frank Sinatra,Easy Listening,23.760


### <font color='purple'>Fin experimento </font>